In [ ]:
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader

In [ ]:
# Load model and tokenizer
model_name = "Henrychur/MMed-Llama-3-8B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
model = model.half()

In [ ]:
# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [ ]:
import pandas as pd

file_path = "Data/PubMedQA_cleaned.json"
QA_data = pd.read_json(file_path)

In [ ]:
# Preprocess the dataset
def preprocess_data(row):
    # Build the input prompt
    input_prompt = f"""Read an abstract from a PubMed paper and answer the question: {row['context']}

Question: {row['question']}
Answer:"""
    # The output is the gold answer (e.g., "No", "Maybe", or "Yes")
    output = row['options'][row['gold_index']]
    return input_prompt, output

# Apply preprocessing
processed_data = QA_data.apply(preprocess_data, axis=1)
inputs, outputs = zip(*processed_data)

# Assign eos_token as pad_token
tokenizer.pad_token = tokenizer.eos_token
# Tokenize the inputs and outputs
input_encodings = tokenizer(list(inputs), padding=True, truncation=True, return_tensors="pt")
output_encodings = tokenizer(list(outputs), padding=True, truncation=True, return_tensors="pt")

In [ ]:
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [ ]:
# Create a custom dataset
class QADataset(Dataset):
    def __init__(self, input_encodings, output_encodings):
        self.input_ids = input_encodings["input_ids"]
        self.attention_mask = input_encodings["attention_mask"]
        self.labels = output_encodings["input_ids"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

# Initialize the dataset and DataLoader
dataset = QADataset(input_encodings, output_encodings)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
# Clear CUDA cache
torch.cuda.empty_cache()

In [ ]:
from torch.cuda.amp import autocast
from torch.amp import GradScaler
from torch.optim import AdamW
from tqdm import tqdm

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-4)

# Initialize GradScaler for mixed precision
scaler = GradScaler()

In [ ]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Training loop
num_epochs = 3  
model.train()

for epoch in range(1, num_epochs + 1):  # Start epoch count from 1 for better readability
    print(f"Epoch {epoch}/{num_epochs}")
    loop = tqdm(dataloader, leave=True, desc=f"Training Epoch {epoch}")
    
    for step, batch in enumerate(loop, start=1):  # Add step count for better tracking
        # Move batch to GPU
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass with mixed precision
        with autocast():  # Enable mixed precision
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        # Backward pass with scaled loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        # Update progress bar with loss
        loop.set_postfix({
            "Step": step,
            "Loss": loss.item()
        })

    # Print epoch summary
    print(f"Epoch {epoch} completed. Final Loss: {loss.item():.4f}")